In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn import metrics
import seaborn as sns

In [2]:
df = pd.read_csv("../Data/banking_cleaned.csv")

In [5]:
df.head()

,amount,oldbalanceOrg,oldbalanceDest,isFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,9839.64,170136.0,0.0,0,0,0,0,1,0
1,1864.28,21249.0,0.0,0,0,0,0,1,0
2,181.00,181.0,0.0,1,0,0,0,0,1
3,181.00,181.0,21182.0,1,0,1,0,0,0
4,11668.14,41554.0,0.0,0,0,0,0,1,0


In [4]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)

#### Splitting Data

In [7]:
y = df["isFraud"]
X = df.drop("isFraud", axis=1)

In [6]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Standardizing Data

In [9]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train) 
scaler.transform(X_train)